In [3]:
import tkinter as tk

class BoardGameGUI:
    def __init__(self, master, rows, cols):
        self.master = master
        self.rows = rows
        self.cols = cols
        self.create_board()

    def create_board(self):
        self.buttons = []
        self.board_state = [[None for _ in range(self.cols)] for _ in range(self.rows)]  # Representação do estado do tabuleiro
        for row in range(self.rows):
            button_row = []
            for col in range(self.cols):
                button = tk.Button(self.master, width=4, height=2, command=lambda r=row, c=col: self.on_button_click(r, c))
                button.grid(row=row, column=col)
                button_row.append(button)
            self.buttons.append(button_row)

    def on_button_click(self, row, col):
        piece = self.board_state[row][col]
        if piece:
            print(f"Peça na posição ({row}, {col}): {piece}")
        else:
            print(f"Nenhuma peça na posição ({row}, {col})")

    def place_piece(self, row, col, piece):
        self.board_state[row][col] = piece
        button = self.buttons[row][col]
        button.config(text=piece)

def main():
    root = tk.Tk()
    root.title("Breakthru")
    rows = 7
    cols = 7
    board_game_gui = BoardGameGUI(root, rows, cols)
    
    # Exemplo de como colocar uma peça no tabuleiro
    board_game_gui.place_piece(3, 4, "P1")  # Coloca uma peça na posição (3, 4)
    root.mainloop()

if __name__ == "__main__":
    main()


Nenhuma peça na posição (2, 4)
Peça na posição (3, 4): P1
Nenhuma peça na posição (5, 4)
Nenhuma peça na posição (6, 6)
